In [ ]:
import subprocess
## want so encode srg(9,4,1,2) in SAT

var_dict = {}
rev_dict = {}
last_id = 1

def add_new_id(x):
    global last_id
    var_dict[x] = str(last_id)
    x = x + '}'
    x = x[0] + '_{' + x[1:]
    rev_dict[last_id] = x
    last_id += 1
    return str(last_id - 1)
    
def add_old_id(x, old_id):
    var_dict[x] = old_id
    
def get_id(x):
    return var_dict[x]

n = 9

triangle = [[set() for j in range(n)] for i in range(n)]


# create a clause for each triangle
# add a dummy variable t_{i,j,k} where it's true if all e_{i,j}, e{i,k}, and e{j,k} are true
for i in range(n):
    for j in range(i + 1, n):
        n_id = add_new_id("e{},{}".format(i,j))
        add_old_id("e{},{}".format(j,i), n_id)
        for k in range(j + 1, n):
            n_id = add_new_id("t{},{},{}".format(i,j,k))
            triangle[i][j].add(n_id)
            triangle[j][k].add(n_id)
            triangle[i][k].add(n_id)
            
clauses = []
for i in range(n):
    for j in range(i + 1, n):
        for k in range(j + 1, n):
            t3 = var_dict["t{},{},{}".format(i,j,k)]
            e1 = var_dict["e{},{}".format(i,j)]
            e2 = var_dict["e{},{}".format(i,k)]
            e3 = var_dict["e{},{}".format(j,k)]
            # if all edges are true, then t3 must be true
            clauses.append(t3 + ' -' + e1 + ' -' + e2 + ' -' + e3)
            # if one edge is false, t3 must be false as well
            clauses.append('-' + t3 + ' ' + e1)
            clauses.append('-' + t3 + ' ' + e2)
            clauses.append('-' + t3 + ' ' + e3)

# if there is an edge eij then at least one triangle must be true       
for i in range(n):
    for j in range(i + 1, n):
        e1 = var_dict["e{},{}".format(i,j)]
        tmp = "-" + e1
        for k in triangle[i][j]:
            tmp += ' ' + k
        clauses.append(tmp)
        

# if there is an edge eij then no two triangles can be true
cnt = 0
for i in range(n):
    for j in range(i + 1, n):
        e1 = var_dict["e{},{}".format(i,j)]
        for k1 in triangle[i][j]:
            for k2 in triangle[i][j]:
                if k2 <= k1:
                    continue
                clauses.append(' -' + e1 + ' -' + k1 + ' -' + k2)




In [ ]:
# now for quadrilateral
from itertools import permutations

quad = [[set() for j in range(n)] for i in range(n)]


# create a clause for each quad
# add a dummy variable q_{i,j,k,t}
for i in range(n):
    for j in range(i + 1, n):
        for k in range(n):
            for t in range(k+1, n):
                if k == i or k == j or t == i or t == j:
                    continue
                n_id = add_new_id("q{},{},{},{}".format(i,j,k,t))
#                 arr = [i, j, k, t]
#                 for perm in permutations(arr):
#                     rev = "q_{}{}{}{}".format(perm[0],perm[1],perm[2],perm[3])
#                     add_old_id(rev, n_id)
                quad[i][j].add(n_id)
    
for i in range(n):
    for j in range(i + 1, n):
        for k in range(n):
            for t in range(k+1, n):
                if k == i or k == j or t == i or t == j:
                    continue
                
                q4 = var_dict["q{},{},{},{}".format(i,j,k,t)]
                e1 = var_dict["e{},{}".format(i,k)]
                e2 = var_dict["e{},{}".format(i,t)]
                e3 = var_dict["e{},{}".format(j,t)]
                e4 = var_dict["e{},{}".format(j,k)]
                
                # if all edges are true, then q4 must be true
                clauses.append(q4 + ' -' + e1 + ' -' + e2 + ' -' + e3 + ' -' + e4)
                # if one edge is false, t3 must be false as well
                clauses.append('-' + q4 + ' ' + e1)
                clauses.append('-' + q4 + ' ' + e2)
                clauses.append('-' + q4 + ' ' + e3)
                clauses.append('-' + q4 + ' ' + e4)
                    

# if there is no edge eij then at least one quadrilateral must be true       
for i in range(n):
    for j in range(i + 1, n):
        e1 = var_dict["e{},{}".format(i,j)]
        tmp = e1
        for k in quad[i][j]:
            tmp += ' ' + k
        clauses.append(tmp)
        

# if there is no edge eij then no two q can be true
cnt = 0
for i in range(n):
    for j in range(i + 1, n):
        e1 = var_dict["e{},{}".format(i,j)]
        for k1 in quad[i][j]:
            for k2 in quad[i][j]:
                if k2 <= k1:
                    continue
                clauses.append(e1 + ' -' + k1 + ' -' + k2)


In [ ]:
cnf_file = "p cnf {} {}\n".format(last_id - 1,len(clauses))

for s in clauses:
    cnf_file = cnf_file + s + ' 0\n'

# print(cnf_file)
print(last_id)
print(len(clauses))
print(84 * 4 + 36 + 36* 7*3 + 36*21*5 + 36 + 36*21*10 )

with open('my_formula.cnf', 'w') as f:
    f.write(cnf_file)
def parse_edge_string(edge_string):
    edge_string = edge_string.strip().strip("e_{").strip("}")
    i, j = map(int, edge_string.split(","))
    return (i, j)

In [ ]:
# Run CaDiCaL with the input file and capture the output and any errors
result = subprocess.run(['/usr/local/bin/cadical'], input=cnf_file.encode('utf-8'), stdout=subprocess.PIPE, stderr=subprocess.PIPE)
if result.returncode != 0:
    error = result.stderr.decode('utf-8')
    print("cadical failed to find a solution with error:", error)
output = result.stdout.decode('utf-8')

v_lines = [line for line in output.split('\n') if line.startswith('v')]
ans = '\n'.join(v_lines)
ans = ans.replace('v','')
ans = ans.replace('\n','')
numbers_list = [int(num) for num in ans.split()]
edges_list = []
for num in numbers_list:
    if num > 0:
        res = rev_dict[num]
        if res[0] == 'e':
            edges_list.append(parse_edge_string(res))
            print(res)

In [ ]:
def graph_to_csv(n, edges):
    header = ";" + ";".join(str(i) for i in range(n)) + "\n"
    row_strings = []
    for i in range(n):
        row = [str(i)] + ["0" if (i, j) not in edges and (j, i) not in edges else "1" for j in range(n)]
        row_strings.append(";".join(row) + "\n")
    csv_content = header + "".join(row_strings)
    with open("graph.csv", "w") as file:
        file.write(csv_content)
n = 10
graph_to_csv(n, edges_list)
